In [4]:
from collections import namedtuple
from abc import ABC, abstractclassmethod
Customer = namedtuple('Customer', 'name fidelity')

class LinItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order: #上下文
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    @abstractclassmethod
    def discount(self, order):
        """返回折扣金额"""

class FidelityPromo(Promotion): #第一个策略
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): #第二个策略
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): #第三个策略
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [5]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LinItem('banana', 4, .5),
        LinItem('apple', 10, 1.5),
        LinItem('watermellon', 5, 5.0)]


In [6]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00

In [7]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90

In [4]:
from collections import namedtuple
Customer = namedtuple('Customer', 'name fidelity')
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product 
        self.quantity = quantity 
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order: # 上下文
    def __init__(self, customer, cart, promotion=None): 
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart) 
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0 
        else:
            discount = self.promotion(self) 
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>' 
        return fmt.format(self.total(), self.due())

def fidelity_promo(order): 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
def bulk_item_promo(order): 
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart} 
    if len(distinct_items) >= 10:
        return order.total() * .07 
    return 0

In [5]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)